<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Performs Research with Multi-Agent Group Chat
regen Version 0.01, see [re-generative.ai](https://)

**Instructions**

make it your own:
1.  copy this notebook (by pressing "File" in the top right and then "SaveCopy to Drive")

power it:
2. get a new OpenAI API key (you can get an Open-AI API key from ***platform***.openai.com, if you don't already have one - make sure that you have at least $1 to use it, check under "Billig")
3.  open "Secrets" from the sidebar (to the left with the key symbol),
4. press "add a new secret" and then enter "openai-api" under Name and your OpenAI API key (that you just got from platform.openai.com under
 "Value" button
5. toggle on/press "Notebook Access"

enter your prompt:
6. press "add a new secret" again and enter your prompt  (your request to the LLM/AI)as a value beneath and name it "prompt" - make sure "Notebook Access" is toggled on/press for this one as well

running the code:
4. press Ctrl + F9 and wait around half a minute (in German: Strg + F9)


scroll down in the notebook/main page and run the code blocks by hovering over the code fields and pressing the "play" button; make sure to wait until it says completed at the bottom - it should usually only need a few seconds each time. (feel free to read the explanations (and feel free to double check with us before - it is a branch of [this](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat/) code by an established company, in case you want to see the full documentation.))
5. see your results at the bottom - make sure to write "continue" (or whatever you think is appropriate) whenever the model asks you to

Enjoy


**Notes**

internal: link to previous [file](https://colab.research.google.com/drive/1b-mX7iEVh-RJGRUGgK8doj5gA90Gq8Zi?pli=1&authuser=2#scrollTo=4yKRjrfbP1bh)


In [ ]:
#installing the model
%%capture --no-stderr
!pip install pyautogen~=0.2.0b4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
import autogen
from google.colab import userdata

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).


You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

In [ ]:
config_list = [
    {
        'model': 'gpt-4',
        'api_key': userdata.get('openai-api'),
    },
]


gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin. Skip any expressions of gratititude.",
   code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor. Skip any expressions of gratititude.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
''',
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code. Skip any expressions of gratititude."""
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist. Skip any expressions of gratititude.
''',
    llm_config=gpt4_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result. Remember that you can install packages in this environment by executing '!pip install <packagename>'. Skip any expressions of gratititude.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)

observer = autogen.AssistantAgent(
    name="Observer",
    system_message="Observer. Your role is to observe the dialogue and pause the dialogue if you think it would be beneficial for the user to get a chance to give more input. Skip any expressions of gratititude.",
    human_input_mode= "CONTINUE",
    llm_config=gpt4_config,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL. Skip any expressions of gratititude.",
    llm_config=gpt4_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic, observer], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

## Start Chat

In [ ]:
user_proxy.initiate_chat(
    manager,
    message=userdata.get('prompt')
,
)

Admin (to chat_manager):

how to affordably soundproof movable walls

--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan:

1. Scientist: Conduct a study to identify the most effective and affordable materials for soundproofing movable walls. This could include materials such as mass loaded vinyl, soundproof foam, or soundproof curtains.

2. Scientist: Test these materials in a controlled environment to determine their effectiveness in soundproofing. This will involve creating a noise source and measuring the sound levels on both sides of the wall with and without the soundproofing material.

3. Engineer: Based on the scientist's findings, design a prototype of a movable wall that incorporates the chosen soundproofing material. This design should take into account the need for the wall to be movable, so the material needs to be lightweight and flexible.

4. Engineer: Build the prototype and test it in a real-world environmen

RateLimitError: ignored